# import packages¶

In [1]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# from jupyterthemes import jtplot
# jtplot.style(theme='gruvboxd')
# jtplot.style(context='talk', fscale=1.4, spines=False, gridlines='--')
# jtplot.style(ticks=True, grid=False, figsize=(6.5, 5))

import pandas as pd
# import numpy as np

def to_datetime(df,cols=[]):
    df[cols] = df[cols].apply(lambda x: pd.to_datetime(x))
    return df

def decompose_datetime(df,cols=[]):
    for col in cols:
        try:
            df[col+'_month'] = df[col].dt.month
            df[col+'_days'] = df[col].dt.day
            df[col+'_dayofweek'] = df[col].dt.dayofweek
        except:
            raise TypeError("The input column [%s] must be pandas datetime format!" % col)
    return df

def get_dummies(df,cols=[]):
    for col in cols:
        dummy_df = pd.get_dummies(df[col],prefix=col,dummy_na=True)
        df = pd.concat([df,dummy_df],axis=1)
    return df

# load data & simple transform

## train.csv & test.csv = df

In [2]:
train_data = pd.read_csv("../train_data/train.csv",na_values="\\N")
train_data.head(10)

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
0,748147,3163926,2018-04-25,2018-05-25,72.1167,2018-05-25,72.1167
1,672952,3698760,2018-06-09,2018-07-09,258.7045,2018-07-08,258.7045
2,404196,2355665,2018-02-18,2018-03-18,307.9270,NaN,NaN
3,342769,1994522,2018-01-13,2018-02-13,252.9809,2018-02-13,252.9809
4,828139,3602352,2018-06-01,2018-07-01,107.6503,2018-06-25,107.6503
5,795130,4745662,2018-10-25,2018-11-25,201.0499,2018-11-09,201.0499
6,95223,3779424,2018-06-19,2018-07-19,3730.9948,2018-07-19,3730.9948
7,115500,2759330,2018-03-25,2018-04-25,1040.5302,2018-04-25,1040.5302
8,515940,2464315,2018-02-28,2018-03-28,133.9311,2018-03-26,133.9311
9,757631,4509276,2018-09-23,2018-10-23,394.3774,NaN,NaN


In [3]:
# train_data.info()
# train_data['user_id'].is_unique # False
# train_data['listing_id'].is_unique # True

In [124]:
test_data = pd.read_csv("../train_data/test.csv",parse_dates=['auditing_date','due_date'])
test_data.head()

,user_id,listing_id,auditing_date,due_date,due_amt
0,498765,5431438,2019-03-12,2019-04-12,138.5903
1,34524,5443211,2019-03-15,2019-04-15,208.0805
2,821741,5461707,2019-03-22,2019-04-22,421.2097
3,263534,5472320,2019-03-26,2019-04-26,212.6537
4,238853,5459750,2019-03-21,2019-04-21,817.4593


### 转换日期

In [5]:
train_data = to_datetime(train_data,cols=['auditing_date','due_date','repay_date'])
# train_data['repay_amt'].fillna(0.0,inplace=True)
train_data.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
0,748147,3163926,2018-04-25,2018-05-25,72.1167,2018-05-25,72.1167
1,672952,3698760,2018-06-09,2018-07-09,258.7045,2018-07-08,258.7045
2,404196,2355665,2018-02-18,2018-03-18,307.9270,NaT,NaN
3,342769,1994522,2018-01-13,2018-02-13,252.9809,2018-02-13,252.9809
4,828139,3602352,2018-06-01,2018-07-01,107.6503,2018-06-25,107.6503


### 还款天数间隔-转换标签

In [6]:
train_data['repay_audit_delta'] = (train_data['repay_date']-train_data['auditing_date']).dt.days
train_data.fillna(value={'repay_amt':0.0,'repay_audit_delta':32},inplace=True)
train_data.drop('repay_date',axis=1,inplace=True)
train_data = train_data[['repay_amt','repay_audit_delta','user_id','listing_id','auditing_date','due_date','due_amt']]
train_data.head()

,repay_amt,repay_audit_delta,user_id,listing_id,auditing_date,due_date,due_amt
0,72.1167,30.0,748147,3163926,2018-04-25,2018-05-25,72.1167
1,258.7045,29.0,672952,3698760,2018-06-09,2018-07-09,258.7045
2,0.0000,32.0,404196,2355665,2018-02-18,2018-03-18,307.9270
3,252.9809,31.0,342769,1994522,2018-01-13,2018-02-13,252.9809
4,107.6503,24.0,828139,3602352,2018-06-01,2018-07-01,107.6503


In [7]:
df = pd.concat([train_data,test_data],sort=False,ignore_index=True)
print(train_data.shape)
print(test_data.shape)
df.shape

(1000000, 7)
(130000, 5)


(1130000, 7)

### 分解时间为月、日、周

In [8]:
df = decompose_datetime(df,cols=['auditing_date','due_date'])
df.head()

,repay_amt,repay_audit_delta,user_id,listing_id,auditing_date,due_date,due_amt,auditing_date_month,auditing_date_days,auditing_date_dayofweek,due_date_month,due_date_days,due_date_dayofweek
0,72.1167,30.0,748147,3163926,2018-04-25,2018-05-25,72.1167,4,25,2,5,25,4
1,258.7045,29.0,672952,3698760,2018-06-09,2018-07-09,258.7045,6,9,5,7,9,0
2,0.0000,32.0,404196,2355665,2018-02-18,2018-03-18,307.9270,2,18,6,3,18,6
3,252.9809,31.0,342769,1994522,2018-01-13,2018-02-13,252.9809,1,13,5,2,13,1
4,107.6503,24.0,828139,3602352,2018-06-01,2018-07-01,107.6503,6,1,4,7,1,6


### 标的id分类[:1] one_hot

In [9]:
df['listing_cat'] = df['listing_id'].astype('str').str.slice(0,1)
df.head()

,repay_amt,repay_audit_delta,user_id,listing_id,auditing_date,due_date,due_amt,auditing_date_month,auditing_date_days,auditing_date_dayofweek,due_date_month,due_date_days,due_date_dayofweek,listing_cat
0,72.1167,30.0,748147,3163926,2018-04-25,2018-05-25,72.1167,4,25,2,5,25,4,3
1,258.7045,29.0,672952,3698760,2018-06-09,2018-07-09,258.7045,6,9,5,7,9,0,3
2,0.0000,32.0,404196,2355665,2018-02-18,2018-03-18,307.9270,2,18,6,3,18,6,2
3,252.9809,31.0,342769,1994522,2018-01-13,2018-02-13,252.9809,1,13,5,2,13,1,1
4,107.6503,24.0,828139,3602352,2018-06-01,2018-07-01,107.6503,6,1,4,7,1,6,3


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130000 entries, 0 to 1129999
Data columns (total 14 columns):
repay_amt                  1000000 non-null float64
repay_audit_delta          1000000 non-null float64
user_id                    1130000 non-null int64
listing_id                 1130000 non-null int64
auditing_date              1130000 non-null datetime64[ns]
due_date                   1130000 non-null datetime64[ns]
due_amt                    1130000 non-null float64
auditing_date_month        1130000 non-null int64
auditing_date_days         1130000 non-null int64
auditing_date_dayofweek    1130000 non-null int64
due_date_month             1130000 non-null int64
due_date_days              1130000 non-null int64
due_date_dayofweek         1130000 non-null int64
listing_cat                1130000 non-null object
dtypes: datetime64[ns](2), float64(3), int64(8), object(1)
memory usage: 120.7+ MB


In [11]:
# df['listing_cat'].value_counts(dropna=False)
df = get_dummies(df,cols=['listing_cat'])
df.head()

,repay_amt,repay_audit_delta,user_id,listing_id,auditing_date,due_date,due_amt,auditing_date_month,auditing_date_days,auditing_date_dayofweek,due_date_month,due_date_days,due_date_dayofweek,listing_cat,listing_cat_1,listing_cat_2,listing_cat_3,listing_cat_4,listing_cat_5,listing_cat_nan
0,72.1167,30.0,748147,3163926,2018-04-25,2018-05-25,72.1167,4,25,2,5,25,4,3,0,0,1,0,0,0
1,258.7045,29.0,672952,3698760,2018-06-09,2018-07-09,258.7045,6,9,5,7,9,0,3,0,0,1,0,0,0
2,0.0000,32.0,404196,2355665,2018-02-18,2018-03-18,307.9270,2,18,6,3,18,6,2,0,1,0,0,0,0
3,252.9809,31.0,342769,1994522,2018-01-13,2018-02-13,252.9809,1,13,5,2,13,1,1,1,0,0,0,0,0
4,107.6503,24.0,828139,3602352,2018-06-01,2018-07-01,107.6503,6,1,4,7,1,6,3,0,0,1,0,0,0


In [12]:
df.info()
# df.to_csv('./df.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130000 entries, 0 to 1129999
Data columns (total 20 columns):
repay_amt                  1000000 non-null float64
repay_audit_delta          1000000 non-null float64
user_id                    1130000 non-null int64
listing_id                 1130000 non-null int64
auditing_date              1130000 non-null datetime64[ns]
due_date                   1130000 non-null datetime64[ns]
due_amt                    1130000 non-null float64
auditing_date_month        1130000 non-null int64
auditing_date_days         1130000 non-null int64
auditing_date_dayofweek    1130000 non-null int64
due_date_month             1130000 non-null int64
due_date_days              1130000 non-null int64
due_date_dayofweek         1130000 non-null int64
listing_cat                1130000 non-null object
listing_cat_1              1130000 non-null uint8
listing_cat_2              1130000 non-null uint8
listing_cat_3              1130000 non-null uint8
listing_cat_

## listing_info.csv 
auditing_date匹配后与train_data中的一致

In [13]:
listing_info = pd.read_csv("../train_data/listing_info.csv",parse_dates=['auditing_date'])
listing_info.head(10)

,user_id,listing_id,auditing_date,term,rate,principal
0,316610,1556649,2017-11-26,9,7.6,4800
1,62002,1556633,2017-11-26,6,7.6,4000
2,192135,1556629,2017-11-26,12,8.0,8660
3,487382,1556628,2017-11-26,9,7.6,4780
4,235186,1556627,2017-11-26,9,7.6,1480
5,320125,1556622,2017-11-26,9,7.6,7750
6,323790,1556621,2017-11-26,9,7.6,1610
7,11641,1556620,2017-11-26,12,8.0,7640
8,308487,1556619,2017-11-26,6,7.6,4740
9,686160,1556615,2017-11-26,6,7.6,5420


In [14]:
listing_info.info()
# listing_info['user_id'].is_unique # False
# listing_info['listing_id'].is_unique # True

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5484891 entries, 0 to 5484890
Data columns (total 6 columns):
user_id          int64
listing_id       int64
auditing_date    datetime64[ns]
term             int64
rate             float64
principal        int64
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 251.1 MB


### 年金现值系数

In [15]:
listing_info['pvi'] = (1-(1+listing_info['rate']*0.01/12)**(-listing_info['term']))/(listing_info['rate']*0.01/12)

In [16]:
listing_info.drop(['user_id','auditing_date'],axis=1,inplace=True)
listing_info.head()

,listing_id,term,rate,principal,pvi
0,1556649,9,7.6,4800,8.721495
1,1556633,6,7.6,4000,5.869215
2,1556629,12,8.0,8660,11.495782
3,1556628,9,7.6,4780,8.721495
4,1556627,9,7.6,1480,8.721495


In [17]:
df = df.merge(listing_info,how='left',on='listing_id')
df.head()

,repay_amt,repay_audit_delta,user_id,listing_id,auditing_date,due_date,due_amt,auditing_date_month,auditing_date_days,auditing_date_dayofweek,...,listing_cat_1,listing_cat_2,listing_cat_3,listing_cat_4,listing_cat_5,listing_cat_nan,term,rate,principal,pvi
0,72.1167,30.0,748147,3163926,2018-04-25,2018-05-25,72.1167,4,25,2,...,0,0,1,0,0,0,9,7.2,630,8.735835
1,258.7045,29.0,672952,3698760,2018-06-09,2018-07-09,258.7045,6,9,5,...,0,0,1,0,0,0,9,7.2,2260,8.735835
2,0.0000,32.0,404196,2355665,2018-02-18,2018-03-18,307.9270,2,18,6,...,0,1,0,0,0,0,9,7.2,2690,8.735835
3,252.9809,31.0,342769,1994522,2018-01-13,2018-02-13,252.9809,1,13,5,...,1,0,0,0,0,0,9,7.2,2210,8.735835
4,107.6503,24.0,828139,3602352,2018-06-01,2018-07-01,107.6503,6,1,4,...,0,0,1,0,0,0,6,8.6,630,5.852330


In [18]:
del train_data,test_data,listing_info

## user_info.csv
因数据插入时间不同而有重复<br>
20171231-20190330<br>
根据insertdate排序，然后根据保留最后一条最新信息

In [19]:
user_info = pd.read_csv("../train_data/user_info.csv",parse_dates=['reg_mon','insertdate'],na_values='\\N')
# user_info.rename(columns={'insertdate':'userinfo_insertdate'},inplace=True)
print(user_info.shape)
user_info.head(10)

(954209, 8)


,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
0,483833,2017-04-01,男,19,c29,c26,c26241,2018-12-11
1,156772,2016-05-01,男,31,c11,c11,c11159,2018-02-13
2,173388,2016-05-01,男,34,c02,c02,c02182,2018-08-21
3,199107,2016-07-01,女,25,c09,c09,c09046,2018-06-05
4,122560,2016-03-01,男,23,c05,c05,c05193,2018-04-02
5,550399,2017-06-01,男,30,c08,c07,c07282,2018-04-19
6,414650,2017-02-01,男,53,c12,c12,NaN,2018-10-16
7,275042,2016-10-01,女,23,c21,c21,c21174,2018-07-10
8,184180,2016-06-01,男,29,c04,c04,c04348,2018-02-08
9,817503,2018-03-01,男,25,c06,c25,c25199,2018-11-27


### insertdate < auditing_date

In [20]:
user_info = user_info.merge(df[['user_id','auditing_date']],how='left',on='user_id')
user_info.shape

(1190050, 9)

In [21]:
time_limit_userinfo = (user_info['insertdate'].isnull())|(user_info['insertdate']<user_info['auditing_date'])
user_info = user_info[time_limit_userinfo]
user_info.shape

(1156075, 9)

### drop_duplicates

In [22]:
user_info.sort_values(['user_id','insertdate'],inplace=True)
user_info.drop_duplicates(subset=['user_id'],keep='last',inplace=True)
user_info.shape

(928195, 9)

### 注册时间长度audit_reg_delta

In [23]:
user_info['audit_reg_delta'] = (user_info['auditing_date']-user_info['reg_mon']).dt.days #衡量用户新旧

### 身份证/手机号省份归属是否一致

In [24]:
user_info['same_province'] = (user_info['cell_province']==user_info['id_province'])
user_info.head()

,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate,auditing_date,audit_reg_delta,same_province
528930,1,2008-03-01,男,35,c22,c31,c31067,2018-12-16,2018-12-17,3943,False
584506,2,2008-03-01,男,46,c09,c09,c09205,2018-10-04,2018-10-05,3870,True
306082,3,2008-03-01,男,35,c22,c05,c05126,2018-06-05,2018-12-08,3934,False
290941,4,2008-04-01,男,36,c17,c17,c17246,2018-01-13,2018-01-14,3575,True
1188818,5,2008-04-01,男,45,c04,c27,c27120,2019-02-17,2019-02-18,3975,False


### gender/cell_province/id_province/same_province one_hot
cell_province和id_province几乎一致，one_hot其一,以same_province补充

In [25]:
user_info = get_dummies(user_info,cols=['gender','cell_province','same_province'])
user_info.shape

(928195, 49)

### id_city

In [26]:
labels,uniques = user_info['id_city'].factorize()
user_info['id_city'] = user_info['id_city'].replace(dict(zip(uniques,labels)))
user_info['id_city'].head()

528930     0.0
584506     1.0
306082     2.0
290941     3.0
1188818    4.0
Name: id_city, dtype: float64

In [27]:
df = pd.merge(df,user_info.drop(['auditing_date','insertdate'],axis=1),how='left',on='user_id')
df.shape

(1130000, 70)

## user_taglist.csv
因数据插入时间不同而有重复<br>
20171231-20190330<br>
根据insertdate排序，然后根据保留最后一条最新信息

In [28]:
user_taglist = pd.read_csv("../train_data/user_taglist.csv",parse_dates = ['insertdate'])
print(user_taglist.shape)
user_taglist.head()

(615160, 3)


,user_id,taglist,insertdate
0,113401,4707|473|3498|4759|1654|298|2869|1164|212|1885...,2018-10-03
1,378358,751|2207|1100|2099|1832|1911|5347|2254|171|360...,2018-11-30
2,434838,877|3795|5628|70|2684|691|719|4228|631|1541|12...,2018-03-25
3,577061,2431|3242|340|1823|4020|4357|164|620|2168|1192...,2018-05-25
4,566753,3980|3125|1819|1333|1177|3972|621|5800|3632|16...,2018-12-02


### insertdate < auditing_date

In [29]:
user_taglist = user_taglist.merge(df[['user_id','auditing_date']],how='left',on='user_id')
user_taglist.shape

(852010, 4)

In [30]:
time_limit_usertag = (user_taglist['insertdate'].isnull())|(user_taglist['insertdate']<user_taglist['auditing_date'])
user_taglist = user_taglist[time_limit_usertag]
user_taglist.shape

(746948, 4)

### drop_duplicates

In [31]:
# user_taglist.rename(columns={'insertdate':'usertag_insertdate'},inplace=True)

user_taglist.sort_values(['user_id','insertdate'],inplace=True)
user_taglist.drop_duplicates(subset=['user_id'],keep='last',inplace=True)
user_taglist.shape

(535380, 4)

In [33]:
# from sklearn.feature_extraction.text import CountVectorizer
# tags = user_taglist['taglist'].astype('str').apply(lambda x: x.strip().replace('|', ' ').strip())
# tag = CountVectorizer().fit_transform(tags)

# from scipy import sparse
# spare_df = sparse.hstack((df[['user_id','age']][:user_taglist.shape[0]].values, tag), format='csr', dtype='float32')

# from sklearn.feature_extraction.text import TfidfVectorizer
# tag_ = TfidfVectorizer().fit_transform(user_taglist['taglist'])

# tag_.shape

In [34]:
# df.to_csv('./df.csv',index=False)

###  标签数量

In [32]:
user_taglist['tag_num'] = user_taglist['taglist'].apply(lambda x: len(x.split('|')) if type(x) is not float else 0) #标签数量，衡量用户的多元属性
user_taglist.head()

,user_id,taglist,insertdate,auditing_date,tag_num
122069,2,2017|4853|3345|1401|320|727|3703|554|1804|2139...,2018-10-04,2018-10-05,54
445632,4,323|683|2250|436|1288|1044|1876|181|357|50|365...,2018-01-13,2018-01-14,169
306326,5,1868|2580|483|1303|904|3604|4028|2049|196|3551...,2019-02-17,2019-02-18,308
185786,7,2005|271|2365|2216|933|1804|1788|719|2572|469|...,2018-06-12,2018-06-13,152
695096,9,5852|3703|287|5762|5539|5465|4207|3574|3519|11...,2018-11-05,2018-11-06,64


### 标签1-9编码

In [33]:
# 分1-9类标签
tag_header = lambda h: h[0] 
tag_headers = lambda x: '|'.join(sorted(set(map(tag_header,x.split("|"))))) if type(x) is not float else None

user_taglist['taglist'] = user_taglist['taglist'].apply(tag_headers)
tag_dummies = user_taglist['taglist'].str.split("|",expand=True)
tag_dummies.head()

,0,1,2,3,4,5,6,7,8
122069,1,2,3,4,5,6,7,8,9
445632,1,2,3,4,5,6,7,8,9
306326,1,2,3,4,5,6,7,8,9
185786,1,2,3,4,5,6,7,8,9
695096,1,2,3,4,5,6,7,8,9


In [34]:
tag_dummies.columns = ["tag_%s" % i for i in range(1,10)]
tag_dummies = (tag_dummies.notnull())*1
tag_dummies.head()

,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
122069,1,1,1,1,1,1,1,1,1
445632,1,1,1,1,1,1,1,1,1
306326,1,1,1,1,1,1,1,1,1
185786,1,1,1,1,1,1,1,1,1
695096,1,1,1,1,1,1,1,1,1


In [35]:
user_taglist = pd.concat([user_taglist,tag_dummies],axis=1)
user_taglist.tail()

,user_id,taglist,insertdate,auditing_date,tag_num,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
784327,928180,1|2|3|4|5|6|7|8|9,2019-03-26,2019-03-27,62,1,1,1,1,1,1,1,1,1
148931,928186,1|2|3|4|5|7|8|9,2019-03-27,2019-03-28,34,1,1,1,1,1,1,1,1,0
645282,928188,1|2|3|4|5|6,2019-03-30,2019-03-31,27,1,1,1,1,1,1,0,0,0
800678,928190,1|2|3|4|6,2019-03-30,2019-03-31,18,1,1,1,1,1,0,0,0,0
199657,928193,1|2|3|4|5|6|7,2019-03-29,2019-03-30,51,1,1,1,1,1,1,1,0,0


In [36]:
user_taglist.drop(['taglist'],axis=1,inplace=True)

In [37]:
df = pd.merge(df,user_taglist.drop(['insertdate','auditing_date'],axis=1),how='left',on='user_id')
df.shape

(1130000, 80)

In [37]:
user_taglist.to_csv('./user_taglist.csv',index=False)

## user_behavior_logs.csv
20170705-20190330

In [2]:
user_behavior = pd.read_csv("../train_data/user_behavior_logs.csv",parse_dates=['behavior_time'])
user_behavior.head(10)

,user_id,behavior_time,behavior_type
0,842439,2018-09-13 23:17:21,3
1,842439,2018-09-13 23:17:21,3
2,905214,2018-09-13 15:19:30,3
3,905214,2018-09-13 15:19:30,3
4,842439,2018-09-13 23:17:21,3
5,842439,2018-09-13 23:17:21,3
6,905214,2018-09-13 15:18:28,3
7,905214,2018-09-13 15:19:30,3
8,48629,2018-09-13 14:22:32,3
9,905214,2018-09-13 15:18:28,3


In [3]:
user_behavior.info()
user_behavior['user_id'].is_unique # False
user_behavior['behavior_type'].value_counts(dropna=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55781271 entries, 0 to 55781270
Data columns (total 3 columns):
user_id          int64
behavior_time    datetime64[ns]
behavior_type    int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.2 GB


1    43469932
3    11303679
2     1007660
Name: behavior_type, dtype: int64

### 行为时刻-夜晚、凌晨、白天
夜晚：19-24<br>
凌晨:0-6<br>
白天：7-18

In [4]:
user_behavior['user_behavior_hour'] = user_behavior['behavior_time'].dt.hour

In [5]:
user_behavior['user_behavior_hour'] = pd.cut(user_behavior['user_behavior_hour'],bins=[-1,6,18,24],labels=['dawn','daytime','night'])

### 分解行为日期为月、日、周

In [6]:
user_behavior = decompose_datetime(user_behavior,cols=['behavior_time'])
user_behavior.head()

,user_id,behavior_time,behavior_type,user_behavior_hour,behavior_time_month,behavior_time_days,behavior_time_dayofweek
0,842439,2018-09-13 23:17:21,3,night,9,13,3
1,842439,2018-09-13 23:17:21,3,night,9,13,3
2,905214,2018-09-13 15:19:30,3,daytime,9,13,3
3,905214,2018-09-13 15:19:30,3,daytime,9,13,3
4,842439,2018-09-13 23:17:21,3,night,9,13,3


### 一天中的同一行为计数

In [7]:
user_behavior['behavior_time'] = user_behavior['behavior_time'].dt.date
behavior_type_count = user_behavior.groupby(['user_id','behavior_time'])['behavior_type'] \
                        .value_counts(dropna=False)
behavior_type_count.name = "same_behavior_count1d"
behavior_type_count = behavior_type_count.reset_index()

user_behavior = pd.merge(user_behavior,behavior_type_count,on=['user_id','behavior_time','behavior_type'])
user_behavior.head()

,user_id,behavior_time,behavior_type,user_behavior_hour,behavior_time_month,behavior_time_days,behavior_time_dayofweek,same_behavior_count1d
0,842439,2018-09-13,3,night,9,13,3,14
1,842439,2018-09-13,3,night,9,13,3,14
2,842439,2018-09-13,3,night,9,13,3,14
3,842439,2018-09-13,3,night,9,13,3,14
4,842439,2018-09-13,3,night,9,13,3,14


In [8]:
user_behavior.shape

(55781271, 8)

In [9]:
user_behavior.drop_duplicates(inplace=True)
user_behavior.shape

(25025256, 8)

In [10]:
user_behavior.to_csv('./user_behavior.csv',index=False)

## user_repay_logs.csv
20170705-20200330

In [38]:
user_repay = pd.read_csv("../train_data/user_repay_logs.csv",na_values="2200-01-01",parse_dates=['due_date','repay_date'])
print(user_repay.shape)
user_repay.head(10)

(18001297, 7)


,user_id,listing_id,order_id,due_date,due_amt,repay_date,repay_amt
0,748483,1858122,6,2018-06-29,528.6365,2018-06-20,528.6365
1,748483,1858122,4,2018-04-29,528.6365,NaT,528.6365
2,748483,1858122,7,2018-07-29,528.6365,2018-06-20,528.6365
3,748483,1858122,5,2018-05-29,528.6365,2018-05-29,528.6365
4,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365
5,748483,1858122,2,2018-02-28,528.6365,2018-02-16,528.6365
6,748483,1858122,3,2018-03-29,528.6365,2018-03-28,528.6365
7,445749,185013,11,2018-03-05,197.9548,2017-05-25,197.9548
8,445749,185013,6,2017-10-05,197.9548,2017-05-25,197.9548
9,445749,185013,10,2018-02-05,197.9548,2017-05-25,197.9548


In [39]:
user_repay.loc[user_repay['repay_date'].isnull(),'repay_amt'] = None
user_repay.head()

,user_id,listing_id,order_id,due_date,due_amt,repay_date,repay_amt
0,748483,1858122,6,2018-06-29,528.6365,2018-06-20,528.6365
1,748483,1858122,4,2018-04-29,528.6365,NaT,NaN
2,748483,1858122,7,2018-07-29,528.6365,2018-06-20,528.6365
3,748483,1858122,5,2018-05-29,528.6365,2018-05-29,528.6365
4,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365


In [40]:
user_repay['user_id'].is_unique # False
user_repay['listing_id'].is_unique # False

False

### 保留第一期数据

In [41]:
user_repay = user_repay.loc[user_repay['order_id']==1,:].reset_index(drop=True)
user_repay.info()
user_repay.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2768985 entries, 0 to 2768984
Data columns (total 7 columns):
user_id       int64
listing_id    int64
order_id      int64
due_date      datetime64[ns]
due_amt       float64
repay_date    datetime64[ns]
repay_amt     float64
dtypes: datetime64[ns](2), float64(2), int64(3)
memory usage: 147.9 MB


,user_id,listing_id,order_id,due_date,due_amt,repay_date,repay_amt
0,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365
1,369368,3900565,1,2018-08-03,385.5078,2018-07-22,385.5078
2,749102,1699160,1,2018-01-11,338.5357,2017-12-11,338.5357
3,385257,2204015,1,2018-03-03,1106.1967,2018-03-02,1106.1967
4,648677,3811960,1,2018-07-23,385.5078,2018-07-04,385.5078


### due_date/repay_date < auditing_date

In [42]:
user_repay = user_repay.merge(df[['user_id','auditing_date']],how='left',on='user_id')
user_repay.shape

(4234690, 8)

In [43]:
time_limit_userrepay1 = ((user_repay['repay_date'].notnull())&(user_repay['repay_date']<user_repay['auditing_date'])) # 历史未逾期，还款在新成交日之前
time_limit_userrepay2 = ((user_repay['repay_date'].isnull())&(user_repay['due_date']<user_repay['auditing_date'])) # 历史逾期，应还款日在新成交日之前
user_repay = user_repay[time_limit_userrepay1|time_limit_userrepay2]
user_repay.shape

(3532703, 8)

### 历史上有无违约

In [44]:
user_repay['default_h'] = (user_repay['repay_date'].isnull())*1

### 历史名义成交日

In [45]:
from pandas.tseries.offsets import DateOffset
user_repay['auditing_date_h'] = user_repay['due_date']-DateOffset(months=1)
user_repay.head()

,user_id,listing_id,order_id,due_date,due_amt,repay_date,repay_amt,auditing_date,default_h,auditing_date_h
0,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365,2018-07-03,0,2017-12-29
1,369368,3900565,1,2018-08-03,385.5078,2018-07-22,385.5078,2018-10-20,0,2018-07-03
2,749102,1699160,1,2018-01-11,338.5357,2017-12-11,338.5357,2018-05-09,0,2017-12-11
3,385257,2204015,1,2018-03-03,1106.1967,2018-03-02,1106.1967,2018-05-07,0,2018-02-03
5,648677,3811960,1,2018-07-23,385.5078,2018-07-04,385.5078,2018-07-08,0,2018-06-23


### 历史距离成交日的还款天数

In [46]:
user_repay['repay_audit_delta_h'] = (user_repay['repay_date']-user_repay['auditing_date_h']).dt.days

### 历史距离应还款日的天数[提前还款的天数]

In [47]:
user_repay['repay_due_delta_h'] = (user_repay['due_date']-user_repay['repay_date']).dt.days

### 分解成交日和还款日

In [48]:
user_repay.rename(columns={'repay_date':'repay_date_h','due_date':'due_date_h'},inplace=True)
user_repay = decompose_datetime(user_repay,cols=['repay_date_h','auditing_date_h'])
user_repay.head()

,user_id,listing_id,order_id,due_date_h,due_amt,repay_date_h,repay_amt,auditing_date,default_h,auditing_date_h,repay_audit_delta_h,repay_due_delta_h,repay_date_h_month,repay_date_h_days,repay_date_h_dayofweek,auditing_date_h_month,auditing_date_h_days,auditing_date_h_dayofweek
0,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365,2018-07-03,0,2017-12-29,30.0,1.0,1.0,28.0,6.0,12,29,4
1,369368,3900565,1,2018-08-03,385.5078,2018-07-22,385.5078,2018-10-20,0,2018-07-03,19.0,12.0,7.0,22.0,6.0,7,3,1
2,749102,1699160,1,2018-01-11,338.5357,2017-12-11,338.5357,2018-05-09,0,2017-12-11,0.0,31.0,12.0,11.0,0.0,12,11,0
3,385257,2204015,1,2018-03-03,1106.1967,2018-03-02,1106.1967,2018-05-07,0,2018-02-03,27.0,1.0,3.0,2.0,4.0,2,3,5
5,648677,3811960,1,2018-07-23,385.5078,2018-07-04,385.5078,2018-07-08,0,2018-06-23,11.0,19.0,7.0,4.0,2.0,6,23,5


In [49]:
# user_repay[['repay_audit_delta_h','repay_due_delta_h']].corr() #0.99

### 历史实际还款天数max、min、mean、meadian、std

In [50]:
# repay_amt_stat = user_repay.groupby(['user_id'])['repay_amt'].agg({'repay_amt_h_mean':'mean','repay_amt_h_p50':'median',
#                                                'repay_amt_h_max':'max','repay_amt_h_min':'min','repay_amt_std':'std'})

repay_audit_delta_stat = user_repay.groupby(['user_id'])['repay_audit_delta_h'].agg({'repay_audit_delta_h_mean':'mean','repay_audit_delta_h_p50':'median',
                                               'repay_audit_delta_h_max':'max','repay_audit_delta_h_min':'min','repay_audit_delta_h_std':'std'})

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """


In [51]:
user_repay = user_repay.merge(repay_audit_delta_stat.fillna(0),how='left',on='user_id')
user_repay.head()

,user_id,listing_id,order_id,due_date_h,due_amt,repay_date_h,repay_amt,auditing_date,default_h,auditing_date_h,...,repay_date_h_days,repay_date_h_dayofweek,auditing_date_h_month,auditing_date_h_days,auditing_date_h_dayofweek,repay_audit_delta_h_mean,repay_audit_delta_h_p50,repay_audit_delta_h_max,repay_audit_delta_h_min,repay_audit_delta_h_std
0,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365,2018-07-03,0,2017-12-29,...,28.0,6.0,12,29,4,27.000000,30.5,31.0,16.0,7.348469
1,369368,3900565,1,2018-08-03,385.5078,2018-07-22,385.5078,2018-10-20,0,2018-07-03,...,22.0,6.0,7,3,1,19.333333,19.0,27.0,12.0,7.505553
2,749102,1699160,1,2018-01-11,338.5357,2017-12-11,338.5357,2018-05-09,0,2017-12-11,...,11.0,0.0,12,11,0,0.000000,0.0,0.0,0.0,0.000000
3,385257,2204015,1,2018-03-03,1106.1967,2018-03-02,1106.1967,2018-05-07,0,2018-02-03,...,2.0,4.0,2,3,5,26.600000,28.0,30.0,19.0,4.393177
4,648677,3811960,1,2018-07-23,385.5078,2018-07-04,385.5078,2018-07-08,0,2018-06-23,...,4.0,2.0,6,23,5,14.305556,17.0,28.0,0.0,10.009346


In [52]:
# user_repay.info()
user_repay['user_id'].is_unique # False
user_repay['listing_id'].is_unique # False
user_repay['due_date_h'].describe()

count                 3532703
unique                    652
top       2018-02-28 00:00:00
freq                    36007
first     2017-07-05 00:00:00
last      2019-04-30 00:00:00
Name: due_date_h, dtype: object

### 删除不要必要的列并去重

In [53]:
user_repay = user_repay.sort_values(['user_id','due_date_h']).drop(
    ['listing_id','order_id','due_date_h','due_amt','repay_date_h','repay_amt','auditing_date','auditing_date_h'],
    axis=1)
user_repay.head()

,user_id,default_h,repay_audit_delta_h,repay_due_delta_h,repay_date_h_month,repay_date_h_days,repay_date_h_dayofweek,auditing_date_h_month,auditing_date_h_days,auditing_date_h_dayofweek,repay_audit_delta_h_mean,repay_audit_delta_h_p50,repay_audit_delta_h_max,repay_audit_delta_h_min,repay_audit_delta_h_std
559336,3,0,29.0,2.0,2.0,3.0,5.0,1,5,4,29.5,29.5,30.0,29.0,0.57735
559337,3,0,29.0,2.0,2.0,3.0,5.0,1,5,4,29.5,29.5,30.0,29.0,0.57735
1999071,3,0,30.0,0.0,7.0,6.0,4.0,6,6,2,29.5,29.5,30.0,29.0,0.57735
1086366,3,0,30.0,0.0,10.0,6.0,5.0,9,6,3,29.5,29.5,30.0,29.0,0.57735
2056776,4,0,28.0,2.0,10.0,23.0,0.0,9,25,0,28.0,28.0,28.0,28.0,0.00000


In [54]:
user_repay = user_repay.drop_duplicates(subset='user_id',keep='last').reset_index(drop=True) #去重保留最新的一条信息
user_repay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 796610 entries, 0 to 796609
Data columns (total 15 columns):
user_id                      796610 non-null int64
default_h                    796610 non-null int32
repay_audit_delta_h          729504 non-null float64
repay_due_delta_h            729504 non-null float64
repay_date_h_month           729504 non-null float64
repay_date_h_days            729504 non-null float64
repay_date_h_dayofweek       729504 non-null float64
auditing_date_h_month        796610 non-null int64
auditing_date_h_days         796610 non-null int64
auditing_date_h_dayofweek    796610 non-null int64
repay_audit_delta_h_mean     796610 non-null float64
repay_audit_delta_h_p50      796610 non-null float64
repay_audit_delta_h_max      796610 non-null float64
repay_audit_delta_h_min      796610 non-null float64
repay_audit_delta_h_std      796610 non-null float64
dtypes: float64(10), int32(1), int64(4)
memory usage: 88.1 MB


In [55]:
# df = pd.read_csv('./df.csv',parse_dates=['reg_mon','due_date','auditing_date','userinfo_insertdate','usertag_insertdate'])

In [56]:
df = df.merge(user_repay,how='left',on='user_id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1130000 entries, 0 to 1129999
Data columns (total 94 columns):
repay_amt                    1000000 non-null float64
repay_audit_delta            1000000 non-null float64
user_id                      1130000 non-null int64
listing_id                   1130000 non-null int64
auditing_date                1130000 non-null datetime64[ns]
due_date                     1130000 non-null datetime64[ns]
due_amt                      1130000 non-null float64
auditing_date_month          1130000 non-null int64
auditing_date_days           1130000 non-null int64
auditing_date_dayofweek      1130000 non-null int64
due_date_month               1130000 non-null int64
due_date_days                1130000 non-null int64
due_date_dayofweek           1130000 non-null int64
listing_cat                  1130000 non-null object
listing_cat_1                1130000 non-null uint8
listing_cat_2                1130000 non-null uint8
listing_cat_3                1

In [57]:
# user_repay.to_csv('./user_repay.csv',index=False)

# clean finall data

## drop

In [58]:
# drop: user_id,listing_id,auditing_date,due_date,reg_mon,gender,same_province
# woe:listing_cat,gender,cell_province,id_province,id_city,same_province--多分类放弃此步
# cut:age
# text:taglist

df.drop(
    ['user_id','listing_id','auditing_date','due_date','reg_mon','gender','same_province'],
    axis=1,inplace=True)

df['age'] = pd.cut(df['age'],bins=[18,20,30,40,50,100],labels=[1,2,3,4,5])
df = pd.get_dummies(df,dummy_na=True,columns=['age'])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1130000 entries, 0 to 1129999
Data columns (total 92 columns):
repay_amt                    1000000 non-null float64
repay_audit_delta            1000000 non-null float64
due_amt                      1130000 non-null float64
auditing_date_month          1130000 non-null int64
auditing_date_days           1130000 non-null int64
auditing_date_dayofweek      1130000 non-null int64
due_date_month               1130000 non-null int64
due_date_days                1130000 non-null int64
due_date_dayofweek           1130000 non-null int64
listing_cat                  1130000 non-null object
listing_cat_1                1130000 non-null uint8
listing_cat_2                1130000 non-null uint8
listing_cat_3                1130000 non-null uint8
listing_cat_4                1130000 non-null uint8
listing_cat_5                1130000 non-null uint8
listing_cat_nan              1130000 non-null uint8
term                         1130000 non-null in

,repay_amt,repay_audit_delta,due_amt,auditing_date_month,auditing_date_days,auditing_date_dayofweek,due_date_month,due_date_days,due_date_dayofweek,listing_cat,...,repay_audit_delta_h_p50,repay_audit_delta_h_max,repay_audit_delta_h_min,repay_audit_delta_h_std,age_1.0,age_2.0,age_3.0,age_4.0,age_5.0,age_nan
0,72.1167,30.0,72.1167,4,25,2,5,25,4,3,...,16.0,30.0,2.0,16.165808,0,1,0,0,0,0
1,258.7045,29.0,258.7045,6,9,5,7,9,0,3,...,29.0,30.0,18.0,5.343554,0,0,1,0,0,0
2,0.0000,32.0,307.9270,2,18,6,3,18,6,2,...,7.0,7.0,7.0,0.000000,0,1,0,0,0,0
3,252.9809,31.0,252.9809,1,13,5,2,13,1,1,...,29.0,30.0,26.0,2.081666,0,1,0,0,0,0
4,107.6503,24.0,107.6503,6,1,4,7,1,6,3,...,30.0,30.0,30.0,0.000000,0,1,0,0,0,0


## correlation

In [59]:
# correlation
corr = df.drop(['repay_amt','repay_audit_delta'],axis=1).corr()
corr.loc[:,:] = np.tril(corr,k=-1)
corr = corr.stack()
pd.DataFrame(corr[corr.abs()>=0.8],columns=["corr"])

corr
due_date_days            auditing_date_days        0.999856
listing_cat_4            due_date_month            0.814556
principal                due_amt                   0.851694
pvi                      term                      0.999860
gender_男                 gender_女                 -1.000000
same_province_True       same_province_False      -1.000000
repay_due_delta_h        repay_audit_delta_h      -0.994617
repay_audit_delta_h_mean repay_audit_delta_h       0.821686
                         repay_due_delta_h        -0.820497
repay_audit_delta_h_p50  repay_audit_delta_h_mean  0.978801
repay_audit_delta_h_max  repay_audit_delta_h_mean  0.874498
                         repay_audit_delta_h_p50   0.865686
repay_audit_delta_h_min  repay_audit_delta_h_mean  0.859800

In [60]:
df.drop(
    ['due_date_days','term','gender_女','same_province_False','repay_due_delta_h','repay_audit_delta_h_p50'],
    axis=1,inplace=True)

In [61]:
df.shape

(1130000, 86)

### generate final train_set / test_set

In [90]:
df.drop(
    ['cell_province','id_province','listing_cat'],axis=1,inplace=True)

In [63]:
# missing_rate
df.apply(lambda x:x.isnull().sum()/len(x))

repay_amt                    0.115044
repay_audit_delta            0.115044
due_amt                      0.000000
auditing_date_month          0.000000
auditing_date_days           0.000000
auditing_date_dayofweek      0.000000
due_date_month               0.000000
due_date_dayofweek           0.000000
listing_cat                  0.000000
listing_cat_1                0.000000
listing_cat_2                0.000000
listing_cat_3                0.000000
listing_cat_4                0.000000
listing_cat_5                0.000000
listing_cat_nan              0.000000
rate                         0.000000
principal                    0.000000
pvi                          0.000000
cell_province                0.019242
id_province                  0.000000
id_city                      0.006961
audit_reg_delta              0.000000
gender_男                     0.000000
gender_nan                   0.000000
cell_province_c01            0.000000
cell_province_c02            0.000000
cell_provinc

In [64]:
df.loc[:,'tag_num':'repay_audit_delta_h_std'] = df.loc[:,'default_h':'repay_audit_delta_h_std'].fillna(-1)

In [79]:
from sklearn.model_selection import StratifiedKFold
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, log_loss, accuracy_score
import time

train_num = 1000000
amt_labels= df.iloc[:train_num,0]
# amt_labels = amt_labels[:train_num]
# clf_labels = clf_labels[:train_num]
clf_labels = df.iloc[:train_num,1]
df = df.iloc[:,2:]
df.fillna(-1,inplace=True)

train_due_amt = df['due_amt']

sub = test_data[['listing_id', 'auditing_date', 'due_amt']]

train_values, test_values = df[:train_num], df[train_num:]


print(train_values.shape)
# 五折验证也可以改成一次验证，按时间划分训练集和验证集，以避免由于时序引起的数据穿越问题。
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
clf = LGBMClassifier(
    learning_rate=0.05,
    n_estimators=10000,
    subsample=0.8,
    subsample_freq=1,
    colsample_bytree=0.8,
    random_state=2019
)
amt_oof = np.zeros(train_num)
prob_oof = np.zeros((train_num, 33))
test_pred_prob = np.zeros((test_values.shape[0], 33))

for i, (trn_idx, val_idx) in enumerate(skf.split(train_values, clf_labels)):
    print(i, 'fold...')
    t = time.time()

    trn_x, trn_y = train_values.loc[trn_idx,:], clf_labels[trn_idx]
    val_x, val_y = train_values.loc[val_idx,:], clf_labels[val_idx]
    val_repay_amt = amt_labels[val_idx]
    val_due_amt = train_due_amt[val_idx]

    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        early_stopping_rounds=1000, verbose=5
    )
    # shepe = (-1, 33)
    val_pred_prob_everyday = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    prob_oof[val_idx] = val_pred_prob_everyday
    val_pred_prob_today = [val_pred_prob_everyday[i][int(val_y.values[i])] for i in range(val_pred_prob_everyday.shape[0])]
    val_pred_repay_amt = val_due_amt.values * val_pred_prob_today
    print('val rmse:', np.sqrt(mean_squared_error(val_repay_amt, val_pred_repay_amt)))
    print('val mae:', mean_absolute_error(val_repay_amt, val_pred_repay_amt))
    amt_oof[val_idx] = val_pred_repay_amt
    test_pred_prob += clf.predict_proba(test_values, num_iteration=clf.best_iteration_) / skf.n_splits

    print('runtime: {}\n'.format(time.time() - t))

print('\ncv rmse:', np.sqrt(mean_squared_error(amt_labels, amt_oof)))
print('cv mae:', mean_absolute_error(amt_labels, amt_oof))
print('cv logloss:', log_loss(clf_labels, prob_oof))
print('cv acc:', accuracy_score(clf_labels, np.argmax(prob_oof, axis=1)))

prob_cols = ['prob_{}'.format(i) for i in range(33)]
for i, f in enumerate(prob_cols):
    sub[f] = test_pred_prob[:, i]
sub_example = pd.read_csv('../train_data/submission/submission.csv', parse_dates=['repay_date'])
sub_example = sub_example.merge(sub, on='listing_id', how='left')

sub_example.head()

sub_example['days'] = (sub_example['repay_date'] - sub_example['auditing_date']).dt.days

test_prob = sub_example[prob_cols].values
test_labels = sub_example['days'].values
test_prob = [test_prob[i][test_labels[i]] for i in range(test_prob.shape[0])]
sub_example['repay_amt'] = sub_example['due_amt'] * test_prob

max_days = np.argmax(sub_example[prob_cols].values,axis=1)

sub_example[['listing_id', 'repay_date', 'repay_amt']].to_csv('./predict.csv', index=False)

In [156]:
sub_example.loc[(max_days==32),['repay_date','repay_amt']] = None

In [159]:
sub_example[['listing_id', 'repay_date', 'repay_amt']].drop_duplicates().to_csv('./predict_null.csv',index=False)